In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io

mat = scipy.io.loadmat('spamData.mat')

mat['Xtrain'] = np.log(mat['Xtrain'] + 0.1)
mat['Xtest'] = np.log(mat['Xtest'] + 0.1)
flattened_ytest = mat['ytest'].flatten() #This section is loading all the necessary data sets from matlab and manipulating some 
#data so they would be easier to deal with when executing the code

##############################################################################################################################
ytrain0 = []
ytrain1 = []
flattened_ytrain = mat['ytrain'].flatten()
count = 0
for i in range(3065):
    if flattened_ytrain[i] == 1:
        count += 1
        ytrain1.append(mat['Xtrain'][i])
    else:
        ytrain0.append(mat['Xtrain'][i]) #split the ytrain datasets to ytrain datapoints that are 1(spam) and 0(non-spam)
        
L_ML = count / 3065
ln_L_ML = np.log(L_ML)

##############################################################################################################################
neat_ytrain1 = np.array(ytrain1).transpose()
neat_ytrain0 = np.array(ytrain0).transpose()

mean_ytrain1 = [sum(item)/len(item) for item in neat_ytrain1] #computing the mean for class 1(spans across 57 features)
mean_ytrain0 = [sum(item)/len(item) for item in neat_ytrain0] #computing the mean for class 0(spans across 57 features)

before_summation = []#From this line onwards, it is the intermediate steps to computing the variance for the 2 classes
before_summation0 = []
for i in range(len(neat_ytrain1)):
    before_summation_value = (neat_ytrain1[i] - mean_ytrain1[i])**2
    before_summation.append(before_summation_value)
    
for i in range(len(neat_ytrain0)):
    before_summation_value0 = (neat_ytrain0[i] - mean_ytrain0[i])**2
    before_summation0.append(before_summation_value0)# up till this line

variance_ytrain1 = [sum(item)/len(item) for item in before_summation] #computing the variance for class 1(spans across 57 features)
variance_ytrain0 = [sum(item)/len(item) for item in before_summation0] #computing the variance for class 0(spans across 57 features)

logpy1_list = []
logpy0_list = []
logpy1_list_train = []
logpy0_list_train = []
for item in mat['Xtest']:
    logpy1 = sum(-(item - mean_ytrain1)**2/(2*np.array(variance_ytrain1)) - np.log(np.sqrt(2*np.pi*np.array(variance_ytrain1))))\
    + ln_L_ML
    logpy1_list.append(logpy1) #computing the posterior for class 1(spam)
    logpy0 = sum(-(item - mean_ytrain0)**2/(2*np.array(variance_ytrain0)) - np.log(np.sqrt(2*np.pi*np.array(variance_ytrain0))))\
    + np.log(1-L_ML)
    logpy0_list.append(logpy0) #computing the posterior class 0(non spam)
    
correct = 0
for i in range(1536):
    if (np.array(logpy1_list)>np.array(logpy0_list)).astype(int)[i] == flattened_ytest[i]:
        correct += 1 #comparing the 2 posteriors coming from 2 different classes and doing the prediction before checking it with 
        #ytest to compute the error rate

        #The above section is trying to compute the test error rate
#################################################################################################################################
        #The below section is trying to compute the training error rate. Note that the computation steps below are generally 
        #similar to the ones above
for item in mat['Xtrain']:
    logpy1 = sum(-(item - mean_ytrain1)**2/(2*np.array(variance_ytrain1)) - np.log(np.sqrt(2*np.pi*np.array(variance_ytrain1))))\
    + ln_L_ML
    logpy1_list_train.append(logpy1)
    logpy0 = sum(-(item - mean_ytrain0)**2/(2*np.array(variance_ytrain0)) - np.log(np.sqrt(2*np.pi*np.array(variance_ytrain0))))\
    + np.log(1-L_ML)
    logpy0_list_train.append(logpy0)
    
correct_train = 0
for i in range(3065):
    if (np.array(logpy1_list_train)>np.array(logpy0_list_train)).astype(int)[i] == flattened_ytrain[i]:
        correct_train += 1

print(1-correct/1536) #printing the test error rate
print(1-correct_train/3065) #printing the training error rate

0.16015625
0.16574225122349107
